In [1]:
import dash
from dash import dcc
import dash_bootstrap_components as dbc
from dash import html,Input, Output, callback
import plotly.express as px
import urllib
import json
import pandas as pd
import plotly.graph_objects as go
import re


app = dash.Dash(__name__, suppress_callback_exceptions=True)


app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        dbc.NavLink('Go to Page 1', href='/page-1'),
        html.Br(),
        dbc.NavLink('Go to Page 2', href='/page-2'),
        html.Br(),
        dbc.NavLink('Go to Page 3', href='/page-3'),
    ]),
    html.Div(id='page-content')
])

#page-1 source
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")


df2 = pd.DataFrame({
    'x': range(10),
    'y': range(10),
    'z': range(10, 0, -1)
})

# page1 layout
layout_page_1 = html.Div(children=[
    html.Div([html.H1('My Dash', style={'textAlign': 'center'})]),
    html.Div([
        dcc.Graph(
            id='example-graph',
            figure=fig,
            style={'width': '50%'}
        ),
        html.Div([
            dcc.Dropdown(
                id='dropdown',
                options=[
                    {'label': 'Accending', 'value': 'y'},
                    {'label': 'Decending', 'value': 'z'},
                ], 
                value='y', 
                style={'marginBottom': '1px','width':'200px'}
            ),
            dcc.Graph(id='graph', style={'width': '100%'}) 
        ], style={'display': 'flex', 'flexDirection': 'column', 'width': '50%','height':'90%'})
    ], style={'display': 'flex'})
])

#page-2 source
url = "https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json"
response = urllib.request.urlopen(url)
data = json.loads(response.read())


#page-2 layout
layout_page_2 =html.Div(
    [
        html.H4("Supply chain of the energy production"),
        dcc.Graph(id="graph2"),
        html.P("Opacity"),
        dcc.Slider(id="slider", min=0, max=1, value=0.2, step=0.1),
    ]
)

#page-3 source
df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

#paege-3 layout
layout_page_3 =html.Div([
    
    html.Div([
        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={'padding': '10px 5px'}),
    
    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),  
    
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),
    
    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


# Callback for navigate to pages
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return layout_page_1
    if pathname == '/page-2':
        return layout_page_2
    if pathname == '/page-3':
        return layout_page_3

    
# Callback for Page-1 dropdown
@app.callback(
    Output(component_id='graph', component_property='figure'),
    [Input(component_id='dropdown', component_property='value')]
)
def update_graph(selected_value):
    return {
        'data': [go.Scatter(x=df2['x'], y=df2[selected_value])]
    }

# Replace opacity
def replace_opacity(color_string, new_opacity):
    if "rgba" in color_string:
        return re.sub(
            r'rgba\((\d+), (\d+), (\d+), [0-9.]+\)',
            lambda match: f"rgba({match.group(1)}, {match.group(2)}, {match.group(3)}, {new_opacity})", 
            color_string
        )
    else:
        return f"rgba(255, 0, 255, {new_opacity})" 


# Callback for Page-2 replace opacity and update figure
@app.callback(
    Output("graph2", "figure"),
    Input("slider", "value"),
)
def display_sankey(opacity):
    opacity = float(opacity)
    node = data["data"][0]["node"]
    node["color"] = [replace_opacity(c, str(opacity)) for c in node["color"]]

    link = data["data"][0]["link"]
    link["color"] = [node["color"][src] for src in link["source"]]

    fig = go.Figure(go.Sankey(link=link, node=node))
    fig.update_layout(font_size=10)
    return fig


# Callback for Page-3 dropdowns change
@callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])
    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')
    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')
    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')
    return fig


def create_time_series(dff, axis_type, title):
    fig = px.scatter(dff, x='Year', y='Value')
    fig.update_traces(mode='lines+markers')
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')
    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)
    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})
    return fig

# Callback for Page-3 hover data change on X graph
@callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_x_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)

# Callback for Page-3 hover data change on Y graph
@callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_y_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

if __name__ == '__main__':
    app.run_server(debug=True)